In [ ]:
import numpy as np
import pandas as pd

# 特征缩放
def feature_scaling(X):
    return X.apply(lambda column: (column - column.mean()) / column.std())

加载数据

In [ ]:
# 加载数据
def load_and_preprocess_data():
    df = pd.read_csv(filepath_or_buffer='dataFile/ex2data1.txt', names=['text1', 'text2', 'lable'])

    X = df.iloc[:, 0:2]
    y = df.iloc[:, 2]
    # X = feature_scaling(X)
    X.insert(0, 'ones', 1)
    X = np.matrix(X)
    y = np.matrix(y).T
    theta = np.matrix(np.zeros(X.shape[1]))
    return X, y, df, theta

sigmoid 函数
$$
\sigma(x) = \frac{1}{1 + e^{-x}}
$$

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

代价函数
$$

J(\theta) = \frac{1}{m} \sum_{i=1}^m \left[ -y^{(i)} \log(h_\theta(x^{(i)})) - (1 - y^{(i)}) \log(1 - h_\theta(x^{(i)})) \right]
$$

In [ ]:
# 计算代价函数
def compute_cost(X, y, theta):
    m = len(y)
    h = sigmoid(np.dot(X, theta.T))
    J = (1 / m) * (np.sum(-np.log(h).T.dot(y) - np.log(1 - h).T.dot(1 - y)))
    return J

逻辑回归模型的假设函数
$$
\begin{align}
h_\theta(x) &= \sigma(\theta^T x) \\
&= \frac{1}{1 + e^{-\theta^T x}}
\end{align}\\
$$

逻辑回归的梯度下降函数
$$
\begin{align}
\theta_j &= \theta_j - \frac{\alpha}{m} \sum_{i=1}^m \left( h_\theta(x^{(i)}) - y^{(i)} \right) x_j^{(i)}
\end{align}
$$

In [1]:
# 梯度下降算法
def gradient_descent(X, y, theta, alpha, iters):
    m = len(y)
    cost = np.zeros(iters)
    for i in range(iters):
        theta = theta - (alpha / m) * np.dot((sigmoid(np.dot(X, theta.T)) - y).T, X)
        cost[i] = compute_cost(X, y, theta)
    return theta, cost